In [ ]:
import numpy as np # numerical coprations and arry handling
import pandas as pd #manages data in dataframe format for results
import time # meaure the exection time of sorting alogrithms
import tensorflow as tf # core library for building abd training the DQn neural network
from tensorflow.keras import layers # provide layers for neural network architecture
import matplotlib.pyplot as plt #  creates plots for visualization
import seaborn as sns # Enhanches plot aesthetics and statistical visulization
from collections import deque # implelment double-ended queue fott the DQN's experience replay memory
import random # Select random datasets during training and evalutions
import os 

# Check for GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("Using GPU with Metal backend (Apple Silicon)")
else:
    print("No GPU found. Running on CPU.")

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Sorting algorithms
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr

def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)

# Utility function to measure execution time
def measure_execution_time(algorithm, arr):
    start_time = time.time()
    algorithm(arr.copy())
    return time.time() - start_time

In [ ]:
def count_inversions(arr):
    if len(arr) <= 1:
        return 0, arr
    mid = len(arr) // 2
    left_inv, left = count_inversions(arr[:mid])
    right_inv, right = count_inversions(arr[mid:])
    merge_inv, merged = merge_and_count(left, right)
    return left_inv + right_inv + merge_inv, merged

def merge_and_count(left, right):
    result = []
    i = j = inv_count = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            inv_count += len(left) - i
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return inv_count, result

def get_dataset_features(arr):
    size = len(arr)
    if size == 0:
        return np.zeros(6, dtype=np.float32)
    size_normalized = size / 2000000
    sortedness = np.sum(np.diff(arr) > 0) / (size - 1) if size > 1 else 1.0
    inv_count, _ = count_inversions(arr)
    max_inversions = size * (size - 1) / 2 if size > 1 else 1
    inversions_normalized = inv_count / max_inversions
    unique_ratio = len(np.unique(arr)) / size
    range_val = (np.max(arr) - np.min(arr)) / 1000 if size > 0 else 0
    clusters = len(np.unique(np.histogram(arr, bins=min(10, size//1000))[0] > 0))
    return np.array([size_normalized, sortedness, inversions_normalized, unique_ratio, range_val, clusters], dtype=np.float32)

dataset_files = []
for folder in ['Data_set_0-63', 'Data_set_0-1000']:
    folder_path = os.path.join(os.getcwd(), folder)
    if os.path.exists(folder_path):
        files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]
        dataset_files.extend(files)
    else:
        print(f"Warning: Folder {folder} not found in {os.getcwd()}")

if not dataset_files:
    raise FileNotFoundError("No .txt files found in Data_set_0-63 or Data_set_0-1000")

random.shuffle(dataset_files)
split_idx = int(0.8 * len(dataset_files))
train_files = dataset_files[:split_idx]
eval_files = dataset_files[split_idx:]

print(f"Training files: {len(train_files)}")
print(f"Evaluation files: {len(eval_files)}")

In [ ]:
# Precompute features and execution times for training files
train_file_features = {}
train_file_times = {algo.__name__: {} for algo in [bubble_sort, merge_sort, quick_sort]}
for dataset_file in train_files:
    with open(dataset_file, 'r') as f:
        line = f.readline()
        arr = list(map(int, line.split()))
    train_file_features[dataset_file] = get_dataset_features(arr)
    for algo in [bubble_sort, merge_sort, quick_sort]:
        train_file_times[algo.__name__][dataset_file] = measure_execution_time(algo, arr)
    print(f"Precomputed for {os.path.basename(dataset_file)}")

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=5000)
        self.gamma = 0.95
        self.epsilon = 1.0``
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.90  # Faster decay
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        model = tf.keras.Sequential([
            layers.Input(shape=(self.state_size,)),
            layers.Dense(512, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        states = np.array([t[0][0] for t in minibatch])
        actions = np.array([t[1] for t in minibatch])
        rewards = np.array([t[2] for t in minibatch])
        next_states = np.array([t[3][0] for t in minibatch])
        dones = np.array([t[4] for t in minibatch])

        targets = self.model.predict(states, verbose=0)
        targets_next = self.target_model.predict(next_states, verbose=0)

        for i in range(batch_size):
            targets[i][actions[i]] = rewards[i] if dones[i] else rewards[i] + self.gamma * np.amax(targets_next[i])

        self.model.fit(states, targets, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self, name):
        self.model.save_weights(name)

    def load(self, name):
        self.model.load_weights(name)

In [ ]:
def train_dqn(episodes, batch_size=32):
    state_size = 6
    action_size = 3
    agent = DQNAgent(state_size, action_size)
    algorithms = [bubble_sort, merge_sort, quick_sort]
    algo_names = ['Bubble Sort', 'Merge Sort', 'Quick Sort']
    results = []

    files_per_epoch = len(train_files)
    for episode in range(episodes):
        if episode % files_per_epoch == 0:
            random.shuffle(train_files)
        file_idx = episode % files_per_epoch
        dataset_file = train_files[file_idx]
        
        state = train_file_features[dataset_file].reshape(1, state_size)
        exec_times = [train_file_times[algo.__name__][dataset_file] for algo in algorithms]
        best_algo_idx = np.argmin(exec_times)
        action = agent.act(state)
        best_time = exec_times[best_algo_idx] if exec_times[best_algo_idx] > 0 else 1e-6
        reward = -(exec_times[action] / best_time)
        reward = np.clip(reward, -10, 0)
        next_state = state
        done = True

        agent.remember(state, action, reward, next_state, done)
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

        if episode % 5 == 0:
            agent.update_target_model()
            if gpus:
                gpu_memory = tf.config.experimental.get_memory_info('GPU:0')
                print(f"Episode {episode}, GPU Memory Used: {gpu_memory['current'] / 1024**2:.2f} MB, Peak: {gpu_memory['peak'] / 1024**2:.2f} MB")

        folder = 'Data_set_0-63' if 'Data_set_0-63' in dataset_file else 'Data_set_0-1000'
        results.append({
            'Episode': episode,
            'Dataset File': dataset_file,
            'Folder': folder,
            'Predicted Algorithm': algo_names[action],
            'Actual Best': algo_names[best_algo_idx],
            'Execution Time': exec_times[action],
            'Best Execution Time': exec_times[best_algo_idx]
        })
        print(f"Episode {episode}/{episodes}, File: {os.path.basename(dataset_file)}, Predicted: {algo_names[action]}, Best: {algo_names[best_algo_idx]}")

    agent.save('dqn_sorting_model_refined.weights.h5')
    return pd.DataFrame(results)

In [ ]:
def evaluate_model():
    state_size = 6
    action_size = 3
    agent = DQNAgent(state_size, action_size)
    agent.load('dqn_sorting_model_refined.weights.h5')
    algorithms = [bubble_sort, merge_sort, quick_sort]
    algo_names = ['Bubble Sort', 'Merge Sort', 'Quick Sort']
    results = []

    for i, dataset_file in enumerate(eval_files):
        with open(dataset_file, 'r') as f:
            line = f.readline()
            arr = list(map(int, line.split()))
        state = get_dataset_features(arr).reshape(1, state_size)
        action = agent.act(state)
        exec_times = [measure_execution_time(algo, arr) for algo in algorithms]
        best_algo_idx = np.argmin(exec_times)
        folder = 'Data_set_0-63' if 'Data_set_0-63' in dataset_file else 'Data_set_0-1000'
        results.append({
            'Test Index': i,
            'Dataset File': dataset_file,
            'Folder': folder,
            'Predicted Algorithm': algo_names[action],
            'Actual Best': algo_names[best_algo_idx],
            'Execution Time': exec_times[action],
            'Best Execution Time': exec_times[best_algo_idx],
            'Correct': algo_names[action] == algo_names[best_algo_idx]
        })
        print(f"Evaluation {i+1}/{len(eval_files)}, File: {os.path.basename(dataset_file)}, Predicted: {algo_names[action]}, Best: {algo_names[best_algo_idx]}")

    return pd.DataFrame(results)

In [ ]:
def plot_results(df_train, df_test):
    plt.figure(figsize=(12, 5))

    # Training plot
    plt.subplot(1, 2, 1)
    plt.plot(df_train['Episode'], df_train['Execution Time'], label='Predicted')
    plt.plot(df_train['Episode'], df_train['Best Execution Time'], label='Best')
    plt.xlabel('Episode')
    plt.ylabel('Execution Time (s)')
    plt.title('Training: Predicted vs Best Execution Time')
    plt.legend()

    # Test accuracy
    plt.subplot(1, 2, 2)
    accuracy = df_test['Correct'].mean() * 100
    plt.bar(['Accuracy'], [accuracy])
    plt.ylim(0, 100)
    plt.title(f'Test Accuracy: {accuracy:.2f}%')
    plt.ylabel('Percentage (%)')

    plt.tight_layout()
    plt.savefig('training_progress.png')
    plt.show()

In [ ]:
episodes = 1000
df_train = train_dqn(episodes)
df_train.to_csv('training_results.csv', index=False)
df_test = evaluate_model()
df_test.to_csv('test_results.csv', index=False)
plot_results(df_train, df_test)